# Centrummaten, Spreidingsmaten en Samenhang

In dit labo bestuderen we hoe we kansverdelingen kunnen karakteriseren met behulp van samenvattende statistieken. We berekenen verwachtingswaarden, varianties, covarianties en correlaties.

**Leerdoelen:**
- Verwachtingswaarde (gemiddelde) berekenen en interpreteren
- Variantie en standaardafwijking als spreidingsmaten gebruiken
- Mediaan, modus en percentielen bepalen
- Covariantie en correlatie berekenen voor lineaire samenhang
- Het verschil begrijpen tussen ongecorreleerd en onafhankelijk

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from scipy import stats

# Set random seed
rng = np.random.default_rng(42)

## Centrummaten

Centrummaten geven het "zwaartepunt" van een kansverdeling aan.

### Verwachtingswaarde (Expected Value)

De **verwachtingswaarde** $\mathbb{E}[x]$ of $\mu$ is het gemiddelde dat we verwachten bij oneindig veel observaties.

- Discreet: $\mathbb{E}[x] = \sum_x x \cdot P(x)$
- Continu: $\mathbb{E}[x] = \int x \cdot p(x) dx$

## ✍️
Bereken de verwachtingswaarde voor een oneerlijke dobbelsteen met de volgende kansverdeling:
- P(1) = 0.10
- P(2) = 0.15
- P(3) = 0.20
- P(4) = 0.10
- P(5) = 0.15
- P(6) = 0.30

## ✍️
Simuleer dobbelsteenworp en toon aan dat het steekproefgemiddelde convergeert naar de verwachtingswaarde (Wet van Grote Aantallen).

### Lineaire transformaties van verwachtingswaarden

De verwachtingswaarde heeft handige eigenschappen bij lineaire transformaties:

$$\mathbb{E}[ax + b] = a \cdot \mathbb{E}[x] + b$$
$$\mathbb{E}[x + y] = \mathbb{E}[x] + \mathbb{E}[y]$$

## ✍️
Verifieer deze eigenschappen numeriek.

### Mediaan en Modus

- **Mediaan**: De waarde waarbij 50% van de kansmassa links en rechts ligt
- **Modus**: De meest waarschijnlijke waarde (hoogste kans bij discreet, maximum van PDF bij continu)

## ✍️
Vergelijk verwachtingswaarde, mediaan en modus voor de volgende drie distributies:
1. **Symmetrische verdeling**: Normale verdeling $N(0, 1)$
2. **Rechts-scheve verdeling**: Log-normale verdeling met parameters $s=0.8$, $\text{scale}=1$
3. **Links-scheve verdeling**: Beta verdeling met parameters $\alpha=8$, $\beta=2$

Bereken voor elke distributie het gemiddelde (mean), de mediaan en de modus, en visualiseer deze.

## Spreidingsmaten

Spreidingsmaten geven aan hoe sterk waarden variëren rond het centrum.

### Variantie en Standaardafwijking

**Variantie**: $\text{Var}(x) = \mathbb{E}[(x - \mu)^2] = \mathbb{E}[x^2] - (\mathbb{E}[x])^2$

**Standaardafwijking**: $\sigma = \sqrt{\text{Var}(x)}$

## ✍️
Bereken de variantie en standaardafwijking voor een eerlijke dobbelsteen.

### Eigenschappen van variantie

$$\text{Var}(ax + b) = a^2 \cdot \text{Var}(x)$$

Voor onafhankelijke variabelen:
$$\text{Var}(x + y) = \text{Var}(x) + \text{Var}(y)$$

## ✍️
Verifieer deze eigenschappen.

### Vergelijking van spreiding tussen verdelingen

## ✍️
Visualiseer normale verdelingen met verschillende standaardafwijkingen (bv. 0.5, 1.0, 2.0 en 3.0) maar hetzelfde gemiddelde (bv. 15).

### Interkwartielafstand (IQR)

De **IQR** is een robuuste maat voor spreiding, minder gevoelig voor uitschieters:

$$\text{IQR} = Q_3 - Q_1 = P_{75} - P_{25}$$

Uitschieters worden vaak gedefinieerd als waarden buiten $[Q_1 - 1.5 \times \text{IQR}, Q_3 + 1.5 \times \text{IQR}]$

## ✍️
Bereken IQR en detecteer uitschieters in een dataset met extreme waarden.

In [ ]:
# Generate data with outliers
normal_data = rng.normal(0, 1, 200)
outliers = np.array([5, 5.5, -4.5, -5, 6])
data = np.concatenate([normal_data, outliers])

## Covariantie en Correlatie

Voor twee random variabelen meten we de **lineaire samenhang**.

### Covariantie

$$\text{Cov}(x, y) = \mathbb{E}[(x - \mathbb{E}[x])(y - \mathbb{E}[y])]$$

- Positief: gelijktijdige afwijkingen hebben dezelfde richting
- Negatief: gelijktijdige afwijkingen hebben tegengestelde richting  
- ~0: geen lineaire samenhang

## ✍️
Gegeven zijn drie datasets:
1. **Positieve correlatie**: $y = 2x + \epsilon$ met $\epsilon \sim N(0, 0.5)$
2. **Negatieve correlatie**: $y = -1.5x + \epsilon$ met $\epsilon \sim N(0, 0.5)$
3. **Geen correlatie**: $x$ en $y$ zijn onafhankelijk normaal verdeeld

Bereken de covariantie voor elk van deze datasets.

In [ ]:
# Generating three datasets with different relationships
n = 1000

# Dataset 1: Positive correlation - y = 2x + noise
x1 = rng.normal(0, 1, n)
y1 = 2 * x1 + rng.normal(0, 0.5, n)

# Dataset 2: Negative correlation - y = -1.5x + noise
x2 = rng.normal(0, 1, n)
y2 = -1.5 * x2 + rng.normal(0, 0.5, n)

# Dataset 3: No correlation - independent
x3 = rng.normal(0, 1, n)
y3 = rng.normal(0, 1, n)

### Correlatie (Pearson)

**Correlatiecoëfficiënt**: Genormaliseerde covariantie

$$\rho_{xy} = \frac{\text{Cov}(x, y)}{\sigma_x \sigma_y}$$

Eigenschappen:
- $-1 \leq \rho \leq 1$
- $|\rho| = 1$: perfecte lineaire relatie
- $\rho = 0$: geen lineaire correlatie
- Schaal-onafhankelijk (in tegenstelling tot covariantie)

## ✍️
Bereken correlaties en vergelijk met covarianties.

### Correlatie bij verschillende sterktes

## ✍️
Gegeven zijn vier datasets met verschillende correlatiesterktes:
- Dataset 1: zeer sterke positieve correlatie (ρ ≈ 0.95)
- Dataset 2: matige positieve correlatie (ρ ≈ 0.7)
- Dataset 3: zwakke positieve correlatie (ρ ≈ 0.3)
- Dataset 4: geen correlatie (ρ ≈ 0)

Visualiseer deze datasets en vergelijk de correlatiecoëfficiënten.

In [ ]:
# Generate data with four specific correlation strengths
n = 500
rho_targets = [0.95, 0.7, 0.3, 0]

datasets = []
for rho_true in rho_targets:
    # Generate correlated data
    x = rng.normal(0, 1, n)
    noise_scale = np.sqrt(1 - rho_true**2) if rho_true < 1 else 0.01
    y = rho_true * x + noise_scale * rng.normal(0, 1, n)

    # Calculate actual correlation and covariance
    rho_actual = np.corrcoef(x, y)[0, 1]
    cov = np.cov(x, y)[0, 1]

    datasets.append({"x": x, "y": y, "rho_target": rho_true, "rho_actual": rho_actual, "cov": cov})

### Covariantie ≈ 0 betekent NIET onafhankelijkheid!

Covariantie/correlatie meet alleen **lineaire** samenhang. Er kan een sterk niet-lineair verband bestaan met correlatie ≈ 0.

## ✍️
Gegeven zijn drie datasets met niet-lineaire relaties maar correlatie ≈ 0:
1. **Circulaire relatie**: $(x, y)$ liggen op concentrische cirkels
2. **Kwadratische relatie**: $y = x^2 + \epsilon$
3. **Absolute waarde**: $y = |x| + \epsilon$

Bereken de correlatie voor elk van deze datasets en observeer dat deze bijna nul is ondanks duidelijke afhankelijkheid.

In [ ]:
# Generate three datasets with non-linear relationships
n = 1000

# Dataset 1: Circular pattern
theta = rng.uniform(0, 2 * np.pi, n)
r = rng.uniform(0.5, 2, n)
x_circle = r * np.cos(theta)
y_circle = r * np.sin(theta)

# Dataset 2: Quadratic relationship y = x²
x_quad = rng.uniform(-3, 3, n)
y_quad = x_quad**2 + rng.normal(0, 0.5, n)

# Dataset 3: Absolute value y = |x|
x_abs = rng.uniform(-3, 3, n)
y_abs = np.abs(x_abs) + rng.normal(0, 0.3, n)

### Covariantiematrix

Voor een vector van random variabelen kunnen we alle covarianties in een matrix samenvatten:

$$\pmb{\Sigma}_{ij} = \text{Cov}(x_i, x_j)$$

- Diagonaal: varianties
- Off-diagonal: covarianties

## ✍️
Bereken en visualiseer een covariantiematrix voor meerdere variabelen.

In [ ]:
# Generate multivariate data with specific covariance structure
n = 1000
mean = [0, 0, 0]
cov_true = np.array(
    [
        [1.0, 0.7, -0.3],  # var(x1), cov(x1,x2), cov(x1,x3)
        [0.7, 1.5, 0.5],  # cov(x2,x1), var(x2), cov(x2,x3)
        [-0.3, 0.5, 0.8],  # cov(x3,x1), cov(x3,x2), var(x3)
    ]
)

data = rng.multivariate_normal(mean, cov_true, n)